In [1]:
# Importing all libraries.
from pylab import *
from netCDF4 import Dataset
%matplotlib inline
import os
import cmocean as cm
from trackeddy.tracking import *
from trackeddy.datastruct import *
from trackeddy.geometryfunc import *
from trackeddy.init import *
from trackeddy.physics import *
from trackeddy.plotfunc import *
import time


In [2]:
# Output data path
outputpath='/g/data/v45/akm157/model_output/mom/mom01v5_kds75/output306/'
# Import SSH values to python environment.
ncfile=Dataset(outputpath+'rregionsouthern_ocean_daily_eta_t.nc')
eta=ncfile.variables['eta_t'][:]*100
# Import geographic coordinates (Lon,Lat)
lon=ncfile.variables['xt_ocean_sub01'][:]
lat=ncfile.variables['yt_ocean_sub01'][:]

In [3]:
# Import SSH 10 yrs mean values to python environment.
ncfile=Dataset('/home/156/jm5970/notebooks/traceddy/data.output/meanssh_10yrs_AEXP.nc')
ssh_mean=squeeze(ncfile.variables['SSH_mean'][:])
# Import geographic coordinates (Lon,Lat)
lon=ncfile.variables['Longitude'][:]
lat=ncfile.variables['Latitude'][:]

In [4]:
#Area in indexes, probably in the future it will be added an option for lon - lat coords.
areamap=array([[0,len(lon)],[0,len(lat)]])

In [5]:
import time

def analyseddyzt(data,x,y,t0,t1,tstep,maxlevel,minlevel,dzlevel,data_meant='',areamap='',mask='',destdir='',physics='',eddycenter='masscenter',checkgauss=True,diagnostics=False,plotdata=False,pprint=False):
    '''
    *************Analys eddy in z and t ***********
    Function to identify each eddy using closed contours, 
    moving in time and contour levels
    Usage:
    
    Example:

    Author: Josue Martinez Moreno, 2017
    '''
    if len(np.shape(data))<3:
        print('If you whant to analyze in time the data need to be 3d [i.e. data(t,x,y)]')
        #return
    if areamap=='':
        areamap=np.array([[0,len(x)],[0,len(y)]])
    if mask == '':
        if len(np.shape(data))<3:
            mask=ma.getmask(data[:,:])
        else:
            mask=ma.getmask(data[0,:,:])
        
    pp =  Printer(); 
    for ii in range(t0,t1,tstep):
        levellist=np.arange(minlevel,maxlevel+dzlevel,dzlevel)
        farlevel=levellist[0]
        if abs(levellist)[0]<abs(levellist)[-1]:
            levellist=np.flipud(levellist)
            farlevel=levellist[0]
        if data_meant=='':
            print('Be sure the data is an anomaly')
            dataanomaly=data[ii,:,:]
        else:
            dataanomaly=data[ii,:,:]-data_meant
        for ll in levellist:
            if minlevel<0 and maxlevel<0:
                levels=[-500,ll]
            elif minlevel>0 and maxlevel>0:
                levels=[ll,500]
            tic=time.time()
            eddies=scan_eddym(dataanomaly,x,y,levels,ii,areamap,mask=mask,destdir=destdir\
                          ,physics=physics,eddycenter=eddycenter,checkgauss=checkgauss\
                          ,diagnostics=diagnostics,plotdata=plotdata,pprint=pprint)
            print('ellapse identification:',time.time()-tic)
            if ll == farlevel:
                eddz = dict_eddyz(ii,ll,farlevel,eddies,diagnostics=diagnostics)
            else:
                tic=time.time()
                eddz = dict_eddyz(ii,ll,farlevel,eddies,eddz,diagnostics=diagnostics)
                print('ellapse dz:',time.time()-tic)
        if ii==0:
            eddytd=dict_eddyt(ii,eddz)
        else:
            tic=time.time()
            eddytd=dict_eddyt(ii,eddz,eddytd) 
            print('ellapse dt:',time.time()-tic)
        pp.timepercentprint(t0,t1,tstep,ii)
    return eddytd

In [6]:
import time

def dict_eddyt(ts,eddys,eddydt=''):
    '''
    Function to create a dictionary with all the eddies.
    All the keys have the following form:
    {eddyn_0:{neddy,time,position,ellipse,contour},eddyn_1:{neddy,time,position,ellipse,contour}}
    Usage:
    
    '''

    #print('eddys',eddys['MinorAxis'])
    
    if ts==0 or eddydt=='':
        #Define the variable
        eddydt={'eddyn_'+str(eddys['EddyN'][0][0]):{'neddy':eddys['EddyN'][0],'time':ts,'position':eddys['Position'][0],\
                'area':eddys['Area'][0],'ellipse':[eddys['Ellipse'][0]],'contour':[eddys['Contour'][0]],\
                'angle':eddys['Angle'][0],'position_eddy':eddys['PositionEllipse'][0],'level':eddys['Level'][0],\
                'majoraxis':eddys['MajorAxis'][0],'minoraxis':eddys['MinorAxis'][0]}}
        #Append all the new data
        for nn in range(1,len(eddys['EddyN'])):
            eddydt['eddyn_'+str(eddys['EddyN'][nn][0])]={'neddy':eddys['EddyN'][nn],'time':ts,'position':eddys['Position'][nn],\
                    'area':eddys['Area'][nn],'ellipse':[eddys['Ellipse'][nn]],'contour':[eddys['Contour'][nn]],\
                    'angle':eddys['Angle'][nn],'position_eddy':eddys['PositionEllipse'][nn],'level':eddys['Level'][nn],\
                    'majoraxis':eddys['MajorAxis'][nn],'minoraxis':eddys['MinorAxis'][nn]}
    else: 
        checklist=[]
        checklist1=[]
        ceddies=[]
        ### I should be able to parallelize this loop, It's the one that is taking more time 
        tic=time.time()
        for key, value in list(eddydt.items()):

            minoraxis= value['minoraxis']
            #print('test',np.shape(minoraxis))
            majoraxis= value['majoraxis']
            number=value['neddy']
            times=value['time']
            position=value['position']
            position_eddy=value['position_eddy']
            ellipse=value['ellipse']
            contour=value['contour']
            level= value['level']
            if isinstance(number, np.float64) or isinstance(number, np.int64):
                number = number
            else: 
                number = number[0]
            
            for nn in range(0,len(eddys['EddyN'])):
                maxlon=eddys['Contour'][nn][0].max()
                maxlone=eddys['Ellipse'][nn][0].max()
                maxlat=eddys['Contour'][nn][1].max()
                maxlate=eddys['Ellipse'][nn][1].max()
                minlon=eddys['Contour'][nn][0].min()
                minlone=eddys['Ellipse'][nn][0].min()
                minlat=eddys['Contour'][nn][1].min()
                minlate=eddys['Ellipse'][nn][1].min()
                area=eddys['Area'][nn]
                angle=eddys['Angle'][nn]                
                if len(np.shape(value['position']))<2:
                    eddyxt0=value['position'][0]
                    eddyyt0=value['position'][1]
                    areae=value['area']
                    anglee=value['angle']
                    timee=value['time']
                else:
                    eddyxt0=value['position'][-1,0]
                    eddyyt0=value['position'][-1,1]
                    areae=value['area'][-1]
                    anglee=value['angle'][-1]
                    timee=value['time'][-1]
                eddyxt1=eddys['Position'][nn][0]
                eddyyt1=eddys['Position'][nn][1]
                
                ################################################################
                #### CHECK WHY THE MINORAXIS Probable bug somewhere else.#######
                ################################################################
                
                ################################################################
                ######## Add an analysis depending the maximum z level #########
                ################################################################
                #print(len(np.shape(minoraxis)),np.shape(minoraxis),np.shape(eddys['MinorAxis'][nn]))
                if len(np.shape(minoraxis))==3:
                    minoraxis=np.squeeze(minoraxis)
                if len(np.shape(majoraxis))==3:
                    majoraxis=np.squeeze(majoraxis)
                
                if (eddyxt1<=maxlon and eddyxt1>=minlon and eddyyt1<=maxlat and eddyyt1>=minlat) and\
                    (eddyxt0<=maxlon and eddyxt0>=minlon and eddyyt0<=maxlat and eddyyt0>=minlat) and\
                    int(timee)!=ts and (ts-int(timee))<5:# and (areae<=area+area/4 and areae>=area-area/4):
                       
                    #print(len(np.shape(minoraxis)),np.shape(minoraxis),np.shape(eddys['MinorAxis'][nn]))
                    eddydt['eddyn_'+str(number)]={'neddy':number,'time':np.vstack((value['time'],ts)),\
                                            'position':np.vstack((position,eddys['Position'][nn])),\
                                            'area':np.vstack((areae,area)),'angle':np.vstack((value['angle'],angle)),\
                                            'ellipse':ellipse+[eddys['Ellipse'][nn]],\
                                            'contour':contour+[eddys['Contour'][nn]],\
                                            'position_eddy':np.vstack((position_eddy,eddys['PositionEllipse'][nn])),\
                                            'level':np.vstack((level,eddys['Level'][nn])),\
                                            'minoraxis':np.vstack((minoraxis,np.squeeze(eddys['MinorAxis'][nn]))),\
                                            'majoraxis':np.vstack((majoraxis,np.squeeze(eddys['MajorAxis'][nn])))}
                    #print(np.shape(np.vstack((minoraxis,eddys['MinorAxis'][nn]))),np.shape(eddys['MinorAxis'][nn]))
                    checklist1.append(eddys['EddyN'][nn][0])
                    checklist.append(number)
            ceddies.append(number)
        print('ellapse dt checking existing eddies:',time.time()-tic)
        counter=0
        checklist2=[]
        checklist3=[]
        tic=time.time()
        for key, value in list(eddydt.items()):
            number=value['neddy']
            if isinstance(number, np.float64) or isinstance(number, np.int64):
                number = number
            else: 
                number = number[0]
            for nn in range(0,len(eddys['EddyN'])):
                number1=eddys['EddyN'][nn]
                
                #print((number!=checklist),(number1!=checklist1))
                if (number!=checklist).all() and (number1!=checklist1).all() and (number!=checklist2).all() and (number1!=checklist3).all():
                    checklist2.append(number)
                    checklist3.append(number1)
                    counter=counter+1
                    #print '*****New Eddy*****'
                    #print(number,checklist)
                    #print('max value number',np.max(ceddies))
                    number=np.max(ceddies)+counter
                    if isinstance(number, np.float64) or isinstance(number, np.int64):
                        number = number
                        #print('numberint',str(number),eddys['EddyN'][nn][0])
                    else: 
                        number = eddys['EddyN'][nn][0]
                    #   print('numberlist',str(number))
                
                    #print(eddydt[:]['neddy'])
                    #print(number)
                    eddydt['eddyn_'+str(number)]={'neddy':number,'time':ts,\
                                        'position':eddys['Position'][nn],'area':eddys['Area'][nn],'angle':eddys['Angle'][nn],\
                                        'position_eddy':eddys['PositionEllipse'][nn],'ellipse':[eddys['Ellipse'][nn]],\
                                        'contour':[eddys['Contour'][nn]],'level':eddys['Level'][nn],\
                                        'minoraxis':eddys['MinorAxis'][nn],'majoraxis':eddys['MajorAxis'][nn]}
                #print('New?',number)
        print('ellapse dt checking NON existing eddies:',time.time()-tic)
        #eddydt={'eddyn'+str(eddys['EddyN']):{'time':ts,'position':eddys['Position'],'ellipse':eddys['Ellipse']\
#                                    ,'contour':eddys['Contour']}}
    #print eddydt
    #print(counter)
    return eddydt

In [7]:
eddytd=analyseddyzt(eta,lon,lat,0,10,1,25,5,5,data_meant=ssh_mean,areamap=areamap,mask=''\
                     ,destdir='',physics='',diagnostics=False,pprint=False)

ellapse identification: 47.58726930618286
ellapse identification: 68.11143159866333
ellapse dz: 0.01923823356628418
ellapse identification: 102.38863801956177
ellapse dz: 0.034468650817871094
ellapse identification: 167.3024640083313
ellapse dz: 0.07897686958312988
ellapse identification: 214.00790905952454
ellapse dz: 0.1409919261932373
 0% [>]10% Time Elapsed: 600 s  ellapse identification: 50.064934730529785
ellapse identification: 73.76570558547974
ellapse dz: 0.02042245864868164
ellapse identification: 113.48919129371643
ellapse dz: 0.04165196418762207
ellapse identification: 148.70344138145447
ellapse dz: 0.07195472717285156
ellapse identification: 205.1604619026184
ellapse dz: 0.1220853328704834
ellapse dt checking existing eddies: 15.02173113822937
ellapse dt checking NON existing eddies: 24.6611008644104
ellapse dt: 39.68426251411438
 0% [=>]20% Time Elapsed: 1231 s  ellapse identification: 51.96434545516968
ellapse identification: 73.33242058753967
ellapse dz: 0.0187702178955

KeyboardInterrupt: 

In [8]:
from joblib import Parallel, delayed
import multiprocessing

def neweddystruct(ts,value,eddys):
    checklist=[]
    checklist1=[]
    ceddies=[]
    minoraxis= value['minoraxis']
    #print('test',np.shape(minoraxis))
    majoraxis= value['majoraxis']
    number=value['neddy']
    time=value['time']
    position=value['position']
    position_eddy=value['position_eddy']
    ellipse=value['ellipse']
    contour=value['contour']
    level= value['level']
    if isinstance(number, np.float64) or isinstance(number, np.int64):
        number = number
    else: 
        number = number[0]
    
    for nn in range(0,len(eddys['EddyN'])):
        maxlon=eddys['Contour'][nn][0].max()
        maxlone=eddys['Ellipse'][nn][0].max()
        maxlat=eddys['Contour'][nn][1].max()
        maxlate=eddys['Ellipse'][nn][1].max()
        minlon=eddys['Contour'][nn][0].min()
        minlone=eddys['Ellipse'][nn][0].min()
        minlat=eddys['Contour'][nn][1].min()
        minlate=eddys['Ellipse'][nn][1].min()
        area=eddys['Area'][nn]
        angle=eddys['Angle'][nn]                
        if len(np.shape(value['position']))<2:
            eddyxt0=value['position'][0]
            eddyyt0=value['position'][1]
            areae=value['area']
            anglee=value['angle']
            timee=value['time']
        else:
            eddyxt0=value['position'][-1,0]
            eddyyt0=value['position'][-1,1]
            areae=value['area'][-1]
            anglee=value['angle'][-1]
            timee=value['time'][-1]
        eddyxt1=eddys['Position'][nn][0]
        eddyyt1=eddys['Position'][nn][1]
                
        ################################################################
        #### CHECK WHY THE MINORAXIS Probable bug somewhere else.#######
        ################################################################
        
        ################################################################
        ######## Add an analysis depending the maximum z level #########
        ################################################################
        #print(len(np.shape(minoraxis)),np.shape(minoraxis),np.shape(eddys['MinorAxis'][nn]))
        if len(np.shape(minoraxis))==3:
            minoraxis=np.squeeze(minoraxis)
        if len(np.shape(majoraxis))==3:
            majoraxis=np.squeeze(majoraxis)
                
        if (eddyxt1<=maxlon and eddyxt1>=minlon and eddyyt1<=maxlat and eddyyt1>=minlat) and\
            (eddyxt0<=maxlon and eddyxt0>=minlon and eddyyt0<=maxlat and eddyyt0>=minlat) and\
            int(timee)!=ts and (ts-int(timee))<5:# and (areae<=area+area/4 and areae>=area-area/4):
                       
            #print(len(np.shape(minoraxis)),np.shape(minoraxis),np.shape(eddys['MinorAxis'][nn]))
            eddydt['eddyn_'+str(number)]={'neddy':number,'time':np.vstack((value['time'],ts)),\
                                        'position':np.vstack((position,eddys['Position'][nn])),\
                                        'area':np.vstack((areae,area)),'angle':np.vstack((value['angle'],angle)),\
                                        'ellipse':ellipse+[eddys['Ellipse'][nn]],\
                                        'contour':contour+[eddys['Contour'][nn]],\
                                        'position_eddy':np.vstack((position_eddy,eddys['PositionEllipse'][nn])),\
                                        'level':np.vstack((level,eddys['Level'][nn])),\
                                        'minoraxis':np.vstack((minoraxis,np.squeeze(eddys['MinorAxis'][nn]))),\
                                        'majoraxis':np.vstack((majoraxis,np.squeeze(eddys['MajorAxis'][nn])))}
            #print(np.shape(np.vstack((minoraxis,eddys['MinorAxis'][nn]))),np.shape(eddys['MinorAxis'][nn]))
            checklist1.append(eddys['EddyN'][nn][0])
            checklist.append(number)
    ceddies.append(number)
    return eddydt,checklist,checklist1,ceddies

def dict_eddyt(ts,eddys,eddydt=''):
    '''
    Function to create a dictionary with all the eddies.
    All the keys have the following form:
    {eddyn_0:{neddy,time,position,ellipse,contour},eddyn_1:{neddy,time,position,ellipse,contour}}
    Usage:
    
    '''

    #print('eddys',eddys['MinorAxis'])
    
    if ts==0 or eddydt=='':
        #Define the variable
        eddydt={'eddyn_'+str(eddys['EddyN'][0][0]):{'neddy':eddys['EddyN'][0],'time':ts,'position':eddys['Position'][0],\
                'area':eddys['Area'][0],'ellipse':[eddys['Ellipse'][0]],'contour':[eddys['Contour'][0]],\
                'angle':eddys['Angle'][0],'position_eddy':eddys['PositionEllipse'][0],'level':eddys['Level'][0],\
                'majoraxis':eddys['MajorAxis'][0],'minoraxis':eddys['MinorAxis'][0]}}
        #Append all the new data
        for nn in range(1,len(eddys['EddyN'])):
            eddydt['eddyn_'+str(eddys['EddyN'][nn][0])]={'neddy':eddys['EddyN'][nn],'time':ts,'position':eddys['Position'][nn],\
                    'area':eddys['Area'][nn],'ellipse':[eddys['Ellipse'][nn]],'contour':[eddys['Contour'][nn]],\
                    'angle':eddys['Angle'][nn],'position_eddy':eddys['PositionEllipse'][nn],'level':eddys['Level'][nn],\
                    'majoraxis':eddys['MajorAxis'][nn],'minoraxis':eddys['MinorAxis'][nn]}
    else: 
        
        ### I should be able to parallelize this loop, It's the one that is taking more time 
        tic=time.time()
        num_cores = multiprocessing.cpu_count()
        eddydt,checklist,checklist1,ceddies = Parallel(n_jobs=num_cores)(delayed(neweddystruct)(ts,value,eddys) for key, value in list(eddydt.items()))
        
            
        print('ellapse dt checking existing eddies:',time.time()-tic)
        counter=0
        checklist2=[]
        checklist3=[]
        tic=time.time()
        for key, value in list(eddydt.items()):
            number=value['neddy']
            if isinstance(number, np.float64) or isinstance(number, np.int64):
                number = number
            else: 
                number = number[0]
            for nn in range(0,len(eddys['EddyN'])):
                number1=eddys['EddyN'][nn]
                
                #print((number!=checklist),(number1!=checklist1))
                if (number!=checklist).all() and (number1!=checklist1).all() and (number!=checklist2).all() and (number1!=checklist3).all():
                    checklist2.append(number)
                    checklist3.append(number1)
                    counter=counter+1
                    #print '*****New Eddy*****'
                    #print(number,checklist)
                    #print('max value number',np.max(ceddies))
                    number=np.max(ceddies)+counter
                    if isinstance(number, np.float64) or isinstance(number, np.int64):
                        number = number
                        #print('numberint',str(number),eddys['EddyN'][nn][0])
                    else: 
                        number = eddys['EddyN'][nn][0]
                    #   print('numberlist',str(number))
                
                    #print(eddydt[:]['neddy'])
                    #print(number)
                    eddydt['eddyn_'+str(number)]={'neddy':number,'time':ts,\
                                        'position':eddys['Position'][nn],'area':eddys['Area'][nn],'angle':eddys['Angle'][nn],\
                                        'position_eddy':eddys['PositionEllipse'][nn],'ellipse':[eddys['Ellipse'][nn]],\
                                        'contour':[eddys['Contour'][nn]],'level':eddys['Level'][nn],\
                                        'minoraxis':eddys['MinorAxis'][nn],'majoraxis':eddys['MajorAxis'][nn]}
                #print('New?',number)
        print('ellapse dt checking NON existing eddies:',time.time()-tic)
        #eddydt={'eddyn'+str(eddys['EddyN']):{'time':ts,'position':eddys['Position'],'ellipse':eddys['Ellipse']\
#                                    ,'contour':eddys['Contour']}}
    #print eddydt
    #print(counter)
    return eddydt


In [9]:
eddytd=analyseddyzt(eta,lon,lat,0,10,1,25,5,5,data_meant=ssh_mean,areamap=areamap,mask=''\
                     ,destdir='',physics='',diagnostics=False,pprint=False)

ellapse identification: 44.54474496841431
ellapse identification: 66.73468327522278
ellapse dz: 0.0169217586517334
ellapse identification: 99.72116279602051
ellapse dz: 0.03689122200012207
ellapse identification: 157.63090991973877
ellapse dz: 0.07299637794494629
ellapse identification: 196.47370982170105
ellapse dz: 0.13280701637268066
 0% [>]10% Time Elapsed: 565 s  ellapse identification: 47.63141369819641
ellapse identification: 73.41025066375732
ellapse dz: 0.018187522888183594
ellapse identification: 100.95725345611572
ellapse dz: 0.03869485855102539
ellapse identification: 141.26049828529358
ellapse dz: 0.06964540481567383
ellapse identification: 204.66581058502197
ellapse dz: 0.12111520767211914


OSError: [Errno 12] Cannot allocate memory